In [1]:
import wget
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Enter NCRB Individual PDF url
pdf_url = 'https://ncrb.gov.in/sites/default/files/adsi_reports_previous_year/Table-3.2_2019_0.pdf'
fileName = pdf_url[67:]
try:
    #Delete the file to redownload it
    os.remove(fileName)
except FileNotFoundError as e:
    #Ignore in case FileNotFound Error occurs
    pass
# Download report
fileName = wget.download(pdf_url,fileName)

In [3]:
import numpy as np
import pandas as pd
import camelot

In [4]:
fileName

'Table-3.2_2019_0.pdf'

In [5]:
tables = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "35.59172859450727,711.2387226293025,568.505718901454,51.33671489643947"
                              ],
                          columns=[
                              "49.0588691437803,147.1766074313409,185.65415185783525,224.1316962843296,262.60924071082394,301.0867851373183,336.67851373182555,370.34636510500815,398.2425848142165,435.7581906300485,469.4260420032311,501.1700161550889,545.4191922455574"
                              ],
                          split_text=True,
                          pages="all"
                         )

In [6]:
tables

<TableList n=7>

In [7]:
def printnAllTables(tables):
    for i in range(len(tables)):
        print("Page No: ",(i+1))
        print(tables[i].df)
        print()

In [8]:
# Incase we're intersted in seeing the original scraped tables.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [9]:
# Removes Garbage Rows that may have been detected Before the 'STATES' row
# AND
# Removes Garbage Rows that may have been detected after the 'TOTAL (ALL INDIA)' row
def clean(table):
    while(table.df.iloc[0,1] != 'STATES'):
        table.df = table.df.iloc[1:]
    while(table.df.iloc[-1,1] != 'TOTAL (ALL INDIA)'):
        table.df = table.df.iloc[:-1]
    # The Return line is unnecessary due to all changes being reflected in the original references
    return table

In [10]:
for each_table in tables:
    clean(each_table)

In [11]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [12]:
tableList = [ each.df.copy() for each in tables]

In [13]:
# List of Column headers as column are nested, i.e exhibit a multi level/index structure
# The multi level structre will be generated and applied in the next 
column_header = [
    ['Financial Problems/Indebtedness','Marriage Related Issues (Total)','Marriage Related Issues(Non-Settlement of Marriage)'],
    ['Marriage Related Issues(Dowry Related Issues)','Marriage Related Issues(Extra Marital Affairs)','Marriage Related Issues(Divorce)'],
    ['Marriage Related Issues (Others)','Property Dispute','Impotency/Infertility'],
    ['Family Problems','Service Related Issues (Total)','Service Related Issues(Physical Fatigue)'],
    ['Service Related Issues (Mental Depression)','Service Related Issues (Promotion Related Issues)','Service Related Issues (Leave Related Issues)'],
    ['Service Related Issues(Other Service Conditions)','Drug Abuse/Alcoholic Addiction','Fall in Social Reputation'],
    ['Other Causes','Causes Not Known','Total']
]
len(column_header)

7

In [14]:
def set_Column(df, title):
    column_list = [
        ("Sl. No.",''),
        ("State/UT",''), 
        (title[0],'Male'),
        (title[0],'Female'),
        (title[0],'Transgender'),
        (title[0],'Total'),
        (title[1],'Male'),
        (title[1],'Female'),
        (title[1],'Transgender'),
        (title[1],'Total'),
        (title[2],'Male'),
        (title[2],'Female'),
        (title[2],'Transgender'),
        (title[2],'Total')
    ]
    df.columns = pd.MultiIndex.from_tuples(column_list)

In [15]:
for i in range(len(tableList)):
    set_Column(tableList[i], column_header[i])
    tableList[i].set_index(["Sl. No.","State/UT"],inplace=True)

In [16]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

# OR Print a simgle one according to your choice
# tables[23].df

In [17]:
# Concat all the tables one after the other, as they all have the same indexes.
final = pd.concat(tableList,axis=1)

In [18]:
# Reset Index before saving file, for better formatting in RAW CSV
final.reset_index().to_csv("Cause - wise Distribution of Suicides in Central Armed Police Force (CAPF) during 2019 as registered by StatesUTs (State & UT-wise).csv",index=False)